In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Setup paths - These will be defined when you run the code
BASE_DIR = os.getcwd()
DES_PATH = os.path.join(BASE_DIR, "des_mri.py")

PATHS = {
    "type1_params": os.path.join(BASE_DIR, "type1_params.csv"),
    "type2_daily": os.path.join(BASE_DIR, "type2_daily_arrivals.csv"),
    "type2_dur": os.path.join(BASE_DIR, "type2_durations_min.csv"),
    "meta": os.path.join(BASE_DIR, "simulation_metadata.csv"),
}

class SimulationConfig:
    """Configuration class for simulation parameters"""
    def __init__(self, 
                 policy: str = "new",
                 slot_type1: int = 35,
                 slot_type2: int = 60,
                 lambda_value: float = 17.0,  # Default value to prevent None errors
                 n_workdays: int = 250,
                 seed: int = 123):
        self.policy = policy
        self.slot_type1 = slot_type1
        self.slot_type2 = slot_type2
        self.lambda_value = lambda_value  # Always has a value
        self.n_workdays = n_workdays
        self.seed = seed
    
    def __str__(self):
        return f"Policy: {self.policy}, λ: {self.lambda_value:.1f}, Slot1: {self.slot_type1}, Slot2: {self.slot_type2}, Workdays: {self.n_workdays}, Seed: {self.seed}"

def load_simulation_module():
    """Load the simulation module with error handling"""
    try:
        import importlib.util
        import sys
        spec = importlib.util.spec_from_file_location("des_mri", DES_PATH)
        des_mri = importlib.util.module_from_spec(spec)
        sys.modules["des_mri"] = des_mri
        spec.loader.exec_module(des_mri)
        print(f"Successfully loaded des_mri module from {DES_PATH}")
        return des_mri
    except Exception as e:
        print(f"Error loading simulation module: {e}")
        print(f"Please ensure des_mri.py exists at: {DES_PATH}")
        raise

def get_original_lambda(des_mri_module):
    """Get original lambda value from input files"""
    try:
        inputs_base = des_mri_module.load_inputs_from_csv(
            type1_params_csv=PATHS["type1_params"],
            type2_daily_arrivals_csv=PATHS["type2_daily"],
            type2_durations_csv=PATHS["type2_dur"],
            metadata_csv=PATHS["meta"],
            seed=42
        )
        original_lambda = inputs_base.lambda_per_day
        print(f"Successfully loaded inputs. Original lambda: {original_lambda:.2f}")
        return original_lambda, inputs_base
    except Exception as e:
        print(f"Error loading input files: {e}")
        print(f"Please ensure all CSV files exist:")
        for key, path in PATHS.items():
            print(f"  {key}: {path}")
        raise

def run_simulation(config: SimulationConfig, base_inputs, des_mri_module) -> Tuple:
    """Run a single simulation with given configuration"""
    try:
        # Create modified inputs using the base inputs structure
        modified_inputs = des_mri_module.Inputs(
            lambda_per_day=config.lambda_value,
            mu_duration_min=base_inputs.mu_duration_min,
            sigma_duration_min=base_inputs.sigma_duration_min,
            type2_daily_arrivals=base_inputs.type2_daily_arrivals,
            type2_durations_min=base_inputs.type2_durations_min,
            slot_minutes_type1=config.slot_type1,
            slot_minutes_type2=config.slot_type2,
            work_minutes=base_inputs.work_minutes,
            seed=config.seed,
        )
        
        # Run simulation
        patients_df, daily_machine_df = des_mri_module.run_simulation(
            inputs=modified_inputs,
            n_workdays=config.n_workdays,
            policy=config.policy
        )
        
        return modified_inputs, patients_df, daily_machine_df
    except Exception as e:
        print(f"Error running simulation with config {config}: {e}")
        raise

def compute_kpis(patients_df: pd.DataFrame, daily_machine_df: pd.DataFrame) -> Dict:
    """Compute the three key KPIs: Waiting time, Utilization, Overtime"""
    try:
        # Waiting Time KPI (95th percentile in working days)
        if len(patients_df) > 0:
            waiting_times = patients_df["wait_workdays"].to_numpy()
            wait_p95 = float(np.quantile(waiting_times, 0.95))
        else:
            wait_p95 = 0.0
        
        # Utilization KPI (average across both machines)
        util_by_machine = daily_machine_df.groupby("machine_id")["utilization"].mean()
        machine_0_util = util_by_machine.get(0, 0.0)
        machine_1_util = util_by_machine.get(1, 0.0)
        avg_utilization = float((machine_0_util + machine_1_util) / 2)
        
        # Overtime KPI (average overtime minutes per day)
        overtime_per_day = daily_machine_df.groupby("day_index")["overtime_min"].sum().to_numpy()
        avg_overtime_min = float(np.mean(overtime_per_day)) if len(overtime_per_day) > 0 else 0.0
        
        return {
            "waiting_time_p95": wait_p95,
            "avg_utilization": avg_utilization,
            "avg_overtime_min": avg_overtime_min,
            "n_patients": len(patients_df)
        }
    except Exception as e:
        print(f"Error computing KPIs: {e}")
        return {
            "waiting_time_p95": 0.0,
            "avg_utilization": 0.0,
            "avg_overtime_min": 0.0,
            "n_patients": 0
        }

def run_baseline_comparison(
    output_dir: str = "./simulation_results",
    n_workdays: int = 250,
    n_seeds: int = 5,
    save_charts: bool = True
) -> pd.DataFrame:
    """
    Run baseline comparison of four scenarios: old/new policy with q90/q95 slots
    
    Returns DataFrame with results for:
    - old policy, q90 slots (35/45 minutes)
    - old policy, q95 slots (40/65 minutes) 
    - new policy, q90 slots (35/45 minutes)
    - new policy, q95 slots (40/65 minutes)
    """
    print("=" * 80)
    print("RUNNING BASELINE COMPARISON")
    print("=" * 80)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output directory: {output_dir}")
    
    # Load simulation module and get original lambda
    des_mri_module = load_simulation_module()
    ORIGINAL_LAMBDA, inputs_base = get_original_lambda(des_mri_module)
    
    # Define baseline scenarios (with explicit slot lengths)
    baseline_scenarios = [
        {"policy": "old", "slot_type1": 35, "slot_type2": 60, "slot_label": "q90"},
        {"policy": "old", "slot_type1": 40, "slot_type2": 65, "slot_label": "q95"},
        {"policy": "new", "slot_type1": 35, "slot_type2": 60, "slot_label": "q90"},
        {"policy": "new", "slot_type1": 40, "slot_type2": 65, "slot_label": "q95"},
    ]
    
    results = []
    total_runs = len(baseline_scenarios) * n_seeds
    run_count = 0
    
    for scenario in baseline_scenarios:
        print(f"\nRunning {scenario['policy']} policy with {scenario['slot_label']} slots ({scenario['slot_type1']}/{scenario['slot_type2']} minutes)...")
        seed_results = []
        
        for seed in range(n_seeds):
            run_count += 1
            print(f"\r  Seed {seed+1}/{n_seeds}...", end="")
            
            # Create config with explicit lambda value
            config = SimulationConfig(
                policy=scenario["policy"],
                slot_type1=scenario["slot_type1"],
                slot_type2=scenario["slot_type2"],
                lambda_value=ORIGINAL_LAMBDA,  # Use original lambda
                n_workdays=n_workdays,
                seed=seed
            )
            
            try:
                _, patients, daily = run_simulation(config, inputs_base, des_mri_module)
                kpis = compute_kpis(patients, daily)
                kpis["seed"] = seed
                seed_results.append(kpis)
            except Exception as e:
                print(f"\n  Error in seed {seed}: {e}")
                continue
        
        if seed_results:
            # Aggregate across seeds
            df_seeds = pd.DataFrame(seed_results)
            avg_kpis = df_seeds[["waiting_time_p95", "avg_utilization", "avg_overtime_min"]].mean().to_dict()
            std_kpis = df_seeds[["waiting_time_p95", "avg_utilization", "avg_overtime_min"]].std().to_dict()
            
            results.append({
                "policy": scenario["policy"],
                "slot_config": scenario["slot_label"],
                "slot_type1": scenario["slot_type1"],
                "slot_type2": scenario["slot_type2"],
                "lambda_value": ORIGINAL_LAMBDA,
                "n_workdays": n_workdays,
                "n_seeds": n_seeds,
                "waiting_time_p95_mean": avg_kpis["waiting_time_p95"],
                "waiting_time_p95_std": std_kpis["waiting_time_p95"],
                "avg_utilization_mean": avg_kpis["avg_utilization"],
                "avg_utilization_std": std_kpis["avg_utilization"],
                "avg_overtime_min_mean": avg_kpis["avg_overtime_min"],
                "avg_overtime_min_std": std_kpis["avg_overtime_min"],
                "n_patients_mean": df_seeds["n_patients"].mean(),
            })
    
    print(f"\n\nBaseline comparison completed! Total simulations: {run_count}")
    
    # Create DataFrame
    results_df = pd.DataFrame(results)
    
    # Save results to CSV
    results_file = os.path.join(output_dir, "baseline_results.csv")
    results_df.to_csv(results_file, index=False)
    print(f"Results saved to: {results_file}")
    
    # Create and save charts
    if save_charts and len(results_df) > 0:
        create_baseline_charts(results_df, output_dir, ORIGINAL_LAMBDA)
    
    return results_df

def create_baseline_charts(results_df: pd.DataFrame, output_dir: str, original_lambda: float):
    """Create baseline comparison charts"""
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f'Baseline KPI Comparison (λ = {original_lambda:.1f} patients/day)', 
                fontsize=16, y=1.02)
    
    # Create scenario labels with slot information
    scenario_labels = []
    colors = []
    wait_means, wait_stds = [], []
    util_means, util_stds = [], []
    overtime_means, overtime_stds = [], []
    
    # Order: old-q90, old-q95, new-q90, new-q95
    for _, row in results_df.sort_values(['policy', 'slot_config']).iterrows():
        scenario_labels.append(f"{row['policy'].upper()} ({row['slot_type1']}/{row['slot_type2']} min)")
        colors.append('#1f77b4' if row['policy'] == 'old' else '#ff7f0e')
        
        wait_means.append(row['waiting_time_p95_mean'])
        wait_stds.append(row['waiting_time_p95_std'])
        util_means.append(row['avg_utilization_mean'])
        util_stds.append(row['avg_utilization_std'])
        overtime_means.append(row['avg_overtime_min_mean'])
        overtime_stds.append(row['avg_overtime_min_std'])
    
    # Plot 1: Waiting Time
    ax = axes[0]
    bars = ax.bar(scenario_labels, wait_means, color=colors, alpha=0.8, 
                 yerr=wait_stds, capsize=5, error_kw={'elinewidth': 1.5, 'capthick': 1.5})
    ax.set_title('Waiting Time (95th percentile)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Days', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3, linestyle='--', axis='y')
    
    # Add value labels
    for bar, mean_val, std_val in zip(bars, wait_means, wait_stds):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height * 1.01,
               f'{mean_val:.1f}±{std_val:.1f}', ha='center', va='bottom', 
               fontsize=10, fontweight='bold')
    
    # Plot 2: Utilization
    ax = axes[1]
    bars = ax.bar(scenario_labels, util_means, color=colors, alpha=0.8,
                 yerr=util_stds, capsize=5, error_kw={'elinewidth': 1.5, 'capthick': 1.5})
    ax.set_title('Average Utilization', fontsize=14, fontweight='bold')
    ax.set_ylabel('Utilization Rate', fontsize=12)
    ax.set_ylim(0, 1)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3, linestyle='--', axis='y')
    
    # Add value labels
    for bar, mean_val, std_val in zip(bars, util_means, util_stds):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height * 1.01,
               f'{mean_val:.3f}±{std_val:.3f}', ha='center', va='bottom', 
               fontsize=10, fontweight='bold')
    
    # Plot 3: Overtime
    ax = axes[2]
    bars = ax.bar(scenario_labels, overtime_means, color=colors, alpha=0.8,
                 yerr=overtime_stds, capsize=5, error_kw={'elinewidth': 1.5, 'capthick': 1.5})
    ax.set_title('Average Overtime', fontsize=14, fontweight='bold')
    ax.set_ylabel('Minutes per day', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3, linestyle='--', axis='y')
    
    # Add value labels
    for bar, mean_val, std_val in zip(bars, overtime_means, overtime_stds):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height * 1.01,
               f'{mean_val:.1f}±{std_val:.1f}', ha='center', va='bottom', 
               fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    
    # Save chart
    chart_file = os.path.join(output_dir, "baseline_kpi_comparison.png")
    plt.savefig(chart_file, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)
    print(f"Baseline chart saved to: {chart_file}")
    
    # Also save a summary table as an image
    create_summary_table_image(results_df, output_dir)

def create_summary_table_image(results_df: pd.DataFrame, output_dir: str):
    """Create an image of the summary table"""
    summary_df = results_df.copy()
    summary_df['Waiting Time'] = summary_df.apply(
        lambda x: f"{x['waiting_time_p95_mean']:.1f} ± {x['waiting_time_p95_std']:.1f}", axis=1)
    summary_df['Utilization'] = summary_df.apply(
        lambda x: f"{x['avg_utilization_mean']:.3f} ± {x['avg_utilization_std']:.3f}", axis=1)
    summary_df['Overtime'] = summary_df.apply(
        lambda x: f"{x['avg_overtime_min_mean']:.1f} ± {x['avg_overtime_min_std']:.1f}", axis=1)
    
    display_df = summary_df[['policy', 'slot_config', 'slot_type1', 'slot_type2', 
                           'Waiting Time', 'Utilization', 'Overtime']]
    display_df.columns = ['Policy', 'Config', 'Slot1', 'Slot2', 
                         'Waiting (days)', 'Utilization', 'Overtime (min/day)']
    
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('tight')
    ax.axis('off')
    
    table = ax.table(cellText=display_df.values,
                     colLabels=display_df.columns,
                     cellLoc='center',
                     loc='center',
                     colColours=['#f2f2f2']*len(display_df.columns))
    
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 1.8)
    
    plt.title('Baseline Results Summary', fontsize=14, fontweight='bold', pad=20)
    
    table_file = os.path.join(output_dir, "baseline_summary_table.png")
    plt.savefig(table_file, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)
    print(f"Summary table saved to: {table_file}")





def run_lambda_sensitivity_analysis(
    lambda_range: List[float] = [14.0, 16.0, 18.0, 20.0],
    policies: List[str] = ["new", "old"],
    slot_type1: int = 40,
    slot_type2: int = 65,
    output_dir: str = "./simulation_results2",
    n_workdays: int = 150,
    n_seeds: int = 3,
    save_charts: bool = True
) -> pd.DataFrame:
    """
    Run sensitivity analysis for arrival rate (lambda) for both policies
    """
    print("=" * 80)
    print("RUNNING ARRIVAL RATE (λ) SENSITIVITY ANALYSIS")
    print("=" * 80)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output directory: {output_dir}")
    print(f"Testing parameters:")
    print(f"  Lambda range: {lambda_range}")
    print(f"  Policies: {policies}")
    print(f"  Slot configuration: {slot_type1}/{slot_type2} minutes")
    print(f"  Workdays per simulation: {n_workdays}")
    print(f"  Seeds per configuration: {n_seeds}")
    
    # Load simulation module
    des_mri_module = load_simulation_module()
    ORIGINAL_LAMBDA, inputs_base = get_original_lambda(des_mri_module)
    
    all_results = []
    total_simulations = len(lambda_range) * len(policies) * n_seeds
    sim_count = 0
    
    for policy in policies:
        print(f"\nAnalyzing {policy.upper()} policy...")
        
        for lambda_val in lambda_range:
            seed_results = []
            
            for seed in range(n_seeds):
                sim_count += 1
                if sim_count % 5 == 0:
                    print(f"\r  Progress: {sim_count}/{total_simulations} simulations", end="")
                
                # Create config with explicit lambda value
                config = SimulationConfig(
                    policy=policy,
                    slot_type1=slot_type1,
                    slot_type2=slot_type2,
                    lambda_value=lambda_val,  # Use the lambda value from range
                    n_workdays=n_workdays,
                    seed=seed
                )
                
                try:
                    _, patients, daily = run_simulation(config, inputs_base, des_mri_module)
                    kpis = compute_kpis(patients, daily)
                    seed_results.append(kpis)
                except Exception as e:
                    print(f"\n  Error for {policy} policy, λ={lambda_val}, seed {seed}: {e}")
                    continue
            
            if seed_results:
                # Aggregate across seeds
                df_seeds = pd.DataFrame(seed_results)
                avg_results = df_seeds[["waiting_time_p95", "avg_utilization", "avg_overtime_min"]].mean().to_dict()
                std_results = df_seeds[["waiting_time_p95", "avg_utilization", "avg_overtime_min"]].std().to_dict()
                
                all_results.append({
                    "policy": policy,
                    "lambda_value": lambda_val,
                    "slot_type1": slot_type1,
                    "slot_type2": slot_type2,
                    "n_workdays": n_workdays,
                    "n_seeds": n_seeds,
                    "waiting_time_p95_mean": avg_results.get("waiting_time_p95", np.nan),
                    "waiting_time_p95_std": std_results.get("waiting_time_p95", np.nan),
                    "avg_utilization_mean": avg_results.get("avg_utilization", np.nan),
                    "avg_utilization_std": std_results.get("avg_utilization", np.nan),
                    "avg_overtime_min_mean": avg_results.get("avg_overtime_min", np.nan),
                    "avg_overtime_min_std": std_results.get("avg_overtime_min", np.nan),
                    "n_seeds_completed": len(seed_results),
                    "n_patients_mean": df_seeds["n_patients"].mean(),
                })
    
    print(f"\n\nLambda sensitivity analysis completed! Total simulations: {sim_count}")
    
    # Create DataFrame
    results_df = pd.DataFrame(all_results)
    
    # Save results to CSV
    results_file = os.path.join(output_dir, "lambda_sensitivity_results.csv")
    results_df.to_csv(results_file, index=False)
    print(f"Results saved to: {results_file}")
    
    # Create and save charts
    if save_charts and len(results_df) > 0:
        create_lambda_sensitivity_charts(results_df, output_dir, slot_type1, slot_type2)
    
    return results_df

def create_lambda_sensitivity_charts(results_df: pd.DataFrame, output_dir: str, slot1: int, slot2: int):
    """Create charts showing lambda sensitivity for both policies"""
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(f'Arrival Rate (λ) Sensitivity\nSlot Configuration: {slot1}/{slot2} minutes', 
                fontsize=16, y=1.05)
    
    # Separate data by policy
    for policy in ['new', 'old']:
        policy_data = results_df[results_df['policy'] == policy].sort_values('lambda_value')
        if len(policy_data) == 0:
            continue
        
        # Plot 1: Waiting Time
        ax = axes[0]
        ax.errorbar(policy_data['lambda_value'], policy_data['waiting_time_p95_mean'],
                   yerr=policy_data['waiting_time_p95_std'],
                   marker='o' if policy == 'new' else 's',
                   linewidth=2, markersize=8, capsize=5,
                   label=f'{policy.upper()} Policy', alpha=0.8)
        ax.set_title('Waiting Time (95th percentile)', fontsize=14, fontweight='bold')
        ax.set_xlabel('λ (arrivals per day)', fontsize=12)
        ax.set_ylabel('Days', fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=11)
        
        # Plot 2: Utilization
        ax = axes[1]
        ax.errorbar(policy_data['lambda_value'], policy_data['avg_utilization_mean'],
                   yerr=policy_data['avg_utilization_std'],
                   marker='o' if policy == 'new' else 's',
                   linewidth=2, markersize=8, capsize=5,
                   label=f'{policy.upper()} Policy', alpha=0.8)
        ax.set_title('Average Utilization', fontsize=14, fontweight='bold')
        ax.set_xlabel('λ (arrivals per day)', fontsize=12)
        ax.set_ylabel('Utilization Rate', fontsize=12)
        ax.set_ylim(0, 1)
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=11)
        
        # Plot 3: Overtime
        ax = axes[2]
        ax.errorbar(policy_data['lambda_value'], policy_data['avg_overtime_min_mean'],
                   yerr=policy_data['avg_overtime_min_std'],
                   marker='o' if policy == 'new' else 's',
                   linewidth=2, markersize=8, capsize=5,
                   label=f'{policy.upper()} Policy', alpha=0.8)
        ax.set_title('Average Overtime', fontsize=14, fontweight='bold')
        ax.set_xlabel('λ (arrivals per day)', fontsize=12)
        ax.set_ylabel('Minutes per day', fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=11)
    
    plt.tight_layout()
    
    lambda_chart_file = os.path.join(output_dir, "lambda_sensitivity_chart.png")
    plt.savefig(lambda_chart_file, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)
    print(f"Lambda sensitivity chart saved to: {lambda_chart_file}")


# ============================================================================
# USAGE EXAMPLES - These are commented out for you to uncomment and run
# ============================================================================

"""
# Example 1: Run baseline comparison only
baseline_results = run_baseline_comparison(
    output_dir="./my_results",
    n_workdays=250,  # Number of workdays to simulate
    n_seeds=5,       # Number of random seeds per scenario
    save_charts=True
)


# Example 2: Run lambda sensitivity analysis for both policies
lambda_results = run_lambda_sensitivity_analysis(
    lambda_range=[14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0],
    policies=["new", "old"],
    slot_type1=35,      # Fixed slot lengths
    slot_type2=45,
    output_dir="./my_results",
    n_workdays=150,
    n_seeds=3,
    save_charts=True
)


"""


'\n# Example 1: Run baseline comparison only\nbaseline_results = run_baseline_comparison(\n    output_dir="./my_results",\n    n_workdays=250,  # Number of workdays to simulate\n    n_seeds=5,       # Number of random seeds per scenario\n    save_charts=True\n)\n\n\n# Example 2: Run lambda sensitivity analysis for both policies\nlambda_results = run_lambda_sensitivity_analysis(\n    lambda_range=[14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0],\n    policies=["new", "old"],\n    slot_type1=35,      # Fixed slot lengths\n    slot_type2=45,\n    output_dir="./my_results",\n    n_workdays=150,\n    n_seeds=3,\n    save_charts=True\n)\n\n\n'

In [2]:
run_baseline_comparison(output_dir="./my_results")

RUNNING BASELINE COMPARISON
Output directory: ./my_results
Successfully loaded des_mri module from C:\Users\Qiyuan\Desktop\Maastricht\des_mri.py
Successfully loaded inputs. Original lambda: 16.95

Running old policy with q90 slots (35/60 minutes)...
  Seed 5/5...
Running old policy with q95 slots (40/65 minutes)...
  Seed 5/5...
Running new policy with q90 slots (35/60 minutes)...
  Seed 5/5...
Running new policy with q95 slots (40/65 minutes)...
  Seed 5/5...

Baseline comparison completed! Total simulations: 20
Results saved to: ./my_results\baseline_results.csv
Baseline chart saved to: ./my_results\baseline_kpi_comparison.png
Summary table saved to: ./my_results\baseline_summary_table.png


,policy,slot_config,slot_type1,slot_type2,lambda_value,n_workdays,n_seeds,waiting_time_p95_mean,waiting_time_p95_std,avg_utilization_mean,avg_utilization_std,avg_overtime_min_mean,avg_overtime_min_std,n_patients_mean
0,old,q90,35,60,16.952381,250,5,33.2,3.898718,0.693795,0.001920,0.364082,0.121378,6753.8
1,old,q95,40,65,16.952381,250,5,72.4,2.190890,0.607695,0.002414,0.000000,0.000000,6755.8
2,new,q90,35,60,16.952381,250,5,1.0,0.000000,0.778592,0.008094,3.901993,0.608549,6759.0
3,new,q95,40,65,16.952381,250,5,1.0,0.000000,0.778596,0.009670,3.459776,0.914382,6768.0


In [3]:
run_lambda_sensitivity_analysis(
    lambda_range=[15, 16, 18, 19],
    policies=["new", "old"]
)

RUNNING ARRIVAL RATE (λ) SENSITIVITY ANALYSIS
Output directory: ./simulation_results2
Testing parameters:
  Lambda range: [15, 16, 18, 19]
  Policies: ['new', 'old']
  Slot configuration: 40/65 minutes
  Workdays per simulation: 150
  Seeds per configuration: 3
Successfully loaded des_mri module from C:\Users\Qiyuan\Desktop\Maastricht\des_mri.py
Successfully loaded inputs. Original lambda: 16.95

Analyzing NEW policy...
  Progress: 10/24 simulations
Analyzing OLD policy...
  Progress: 20/24 simulations

Lambda sensitivity analysis completed! Total simulations: 24
Results saved to: ./simulation_results2\lambda_sensitivity_results.csv
Lambda sensitivity chart saved to: ./simulation_results2\lambda_sensitivity_chart.png


,policy,lambda_value,slot_type1,slot_type2,n_workdays,n_seeds,waiting_time_p95_mean,waiting_time_p95_std,avg_utilization_mean,avg_utilization_std,avg_overtime_min_mean,avg_overtime_min_std,n_seeds_completed,n_patients_mean
0,new,15,40,65,150,3,1.0,0.000000,0.726706,0.003912,1.725345,0.412823,3,3745.333333
1,new,16,40,65,150,3,1.0,0.000000,0.750845,0.004139,2.328160,0.311373,3,3914.333333
2,new,18,40,65,150,3,1.0,0.000000,0.803187,0.009575,5.785901,0.706721,3,4218.000000
3,new,19,40,65,150,3,1.0,0.000000,0.822116,0.006228,7.132512,0.369673,3,4377.666667
4,old,15,40,65,150,3,33.0,1.000000,0.602389,0.002772,0.000000,0.000000,3,3760.333333
5,old,16,40,65,150,3,37.0,1.000000,0.604681,0.003453,0.000000,0.000000,3,3934.000000
6,old,18,40,65,150,3,56.0,1.732051,0.606843,0.003117,0.000000,0.000000,3,4219.666667
7,old,19,40,65,150,3,67.0,3.605551,0.604451,0.002491,0.000000,0.000000,3,4372.666667
